In [1]:
%load_ext rpy2.ipython

AA-GP: MultiBLUP results
================
08 October, 2019

In [2]:
%%R
knitr::opts_knit$set(root.dir = "~/Documents/AA-GenomicPrediction",
                     warning = FALSE)
knitr::opts_chunk$set(fig.width = 10)

library(tidyverse)

R[write to console]: Error in loadNamespace(name) : there is no package called ‘knitr’
Calls: <Anonymous> ... tryCatch -> tryCatchList -> tryCatchOne -> <Anonymous>




Error in loadNamespace(name) : there is no package called ‘knitr’
Calls: <Anonymous> ... tryCatch -> tryCatchList -> tryCatchOne -> <Anonymous>


In [3]:
## ── Attaching packages ─────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──

## ✔ ggplot2 3.2.0     ✔ purrr   0.3.2
## ✔ tibble  2.1.3     ✔ dplyr   0.8.3
## ✔ tidyr   1.0.0     ✔ stringr 1.4.0
## ✔ readr   1.3.1     ✔ forcats 0.4.0

## ── Conflicts ────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
## ✖ dplyr::filter() masks stats::filter()
## ✖ dplyr::lag()    masks stats::lag()

In [4]:
%%R
library(broom)
library(quantreg)

R[write to console]: Error in library(broom) : there is no package called ‘broom’
Calls: <Anonymous> -> <Anonymous> -> withVisible -> library




Error in library(broom) : there is no package called ‘broom’
Calls: <Anonymous> -> <Anonymous> -> withVisible -> library


In [5]:
## Loading required package: SparseM

## 
## Attaching package: 'SparseM'

## The following object is masked from 'package:base':
## 
##     backsolve

In [6]:
%%R
library(kableExtra)

R[write to console]: Error in library(kableExtra) : there is no package called ‘kableExtra’
Calls: <Anonymous> -> <Anonymous> -> withVisible -> library




Error in library(kableExtra) : there is no package called ‘kableExtra’
Calls: <Anonymous> -> <Anonymous> -> withVisible -> library


In [7]:
## 
## Attaching package: 'kableExtra'

## The following object is masked from 'package:dplyr':
## 
##     group_rows

## Determine MultiBLUP Thresholds

In [8]:
%%R
load("reports/model_summary.RData")
load("reports/gblup.RData")
load("reports/multiblup.RData")
load("reports/null_distribution.RData")

aa_fam <- unique((gblup_pa$trait)) %>%
  as_tibble() %>%
  mutate(
         # add amino acid biochemical family ids (based on precursor)
         family = ifelse(grepl("asp|thr|met|asp|AspFam", value) & !grepl("ile", value), 
                         "aspartate", 
                         ifelse(grepl("arg|glu|gln|pro|his|GluFam", value),
                                "glutamate",
                                ifelse(grepl("gly|ser|SerFam", value),
                                       "serine",
                                       ifelse(grepl("phe|tyr|trp|ShikFam", value),
                                              "aromatic",
                                              ifelse(grepl("Total", value),
                                                     "total", "BCAA_pyruvate"))))),
         # add type of measurement (absolute, relative, ratio)
         aa_cat = ifelse(!grepl("_", value), "absolute",
                         ifelse(grepl("_t", value), "relative",
                                "family"))) %>%
  rename(trait = "value")

R[write to console]: Error in readChar(con, 5L, useBytes = TRUE) : cannot open the connection
Calls: <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar



R[write to console]: In addition: 


R[write to console]: Warning message:



R[write to console]: In readChar(con, 5L, useBytes = TRUE) :


R[write to console]: 
 


R[write to console]:  cannot open compressed file 'reports/model_summary.RData', probable reason 'No such file or directory'




Error in readChar(con, 5L, useBytes = TRUE) : cannot open the connection
Calls: <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar


In [9]:
## Warning: Calling `as_tibble()` on a vector is discouraged, because the behavior is likely to change in the future. Use `tibble::enframe(name = NULL)` instead.
## This warning is displayed once per session.

In [10]:
%%R
head(aa_fam)

R[write to console]: Error in head(aa_fam) : object 'aa_fam' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head




Error in head(aa_fam) : object 'aa_fam' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head


<div data-pagedtable="false">

<script data-pagedtable-source type="application/json">
{"columns":[{"label":["trait"],"name":[1],"type":["chr"],"align":["left"]},{"label":["family"],"name":[2],"type":["chr"],"align":["left"]},{"label":["aa_cat"],"name":[3],"type":["chr"],"align":["left"]}],"data":[{"1":"ala_PyrFam","2":"BCAA_pyruvate","3":"family"},{"1":"ala_t","2":"BCAA_pyruvate","3":"relative"},{"1":"ala","2":"BCAA_pyruvate","3":"absolute"},{"1":"arg_GluFam","2":"glutamate","3":"family"},{"1":"arg_t","2":"glutamate","3":"relative"},{"1":"arg","2":"glutamate","3":"absolute"}],"options":{"columns":{"min":{},"max":[10]},"rows":{"min":[10],"max":[10]},"pages":{}}}
  </script>

</div>

In [11]:
%%R
# set pathways to exclude
exclude <-
  c(
    "aa_deg_bcat",
    "aa_syn_nobcat",
    "aa_deg_old",
    "aa_syn_old",
    "asp_protease",
    "cys_protease",
    "degradation_AAA",
    "degradation_autophagy",
    "degradation_subtilases",
    "glycolysis_cystolic",
    "glycolysis_plastid",
    "glycolysis_other",
    "metallo_protease",
    "protein_degradation_old",
    "ser_protease"
  )

pathways <- pathways %>% 
  filter(!pathway %in% exclude)

multiblup_thresholds <- pathways %>%
  group_by(pathway) %>%
  mutate(stat = list(c(h2_95 = as.data.frame(
    lapply(h2_fit,
           function(x)
             tryCatch({
               predict(x,
                       data.frame(size = size))
             }, error = function(err) {
               NA
             }))
  ),
  lr_95 = as.data.frame(
    lapply(lr_fit,
           function(x)
             tryCatch({
               predict(x,
                       data.frame(size = size))
             }, error = function(err) {
               NA
             })))))
  ) %>%
  mutate(stat = map(stat, bind_rows)) %>%
  unnest() %>%
  gather(., id, value, -pathway, -n_genes, -size) %>%
  separate(col = id, into = c("stat", "trait"), sep = "\\.") %>%
  spread(., stat, value)

R[write to console]: Error in pathways %>% filter(!pathway %in% exclude) : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in pathways %>% filter(!pathway %in% exclude) : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [12]:
## Warning: `cols` is now required.
## Please use `cols = c(stat)`

In [13]:
%%R
head(multiblup_thresholds)

R[write to console]: Error in head(multiblup_thresholds) : 
  object 'multiblup_thresholds' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head




Error in head(multiblup_thresholds) : 
  object 'multiblup_thresholds' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head


<div data-pagedtable="false">

<script data-pagedtable-source type="application/json">
{"columns":[{"label":["pathway"],"name":[1],"type":["chr"],"align":["left"]},{"label":["n_genes"],"name":[2],"type":["int"],"align":["right"]},{"label":["size"],"name":[3],"type":["int"],"align":["right"]},{"label":["trait"],"name":[4],"type":["chr"],"align":["left"]},{"label":["h2_95"],"name":[5],"type":["dbl"],"align":["right"]},{"label":["lr_95"],"name":[6],"type":["dbl"],"align":["right"]}],"data":[{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"ala","5":"0.226292","6":"3.5832"},{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"ala_PyrFam","5":"0.104866","6":"13.4868"},{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"ala_t","5":"0.252137","6":"4.1898"},{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"arg","5":"0.205128","6":"3.0680"},{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"arg_GluFam","5":"0.471608","6":"3.3996"},{"1":"aa_deg_bcat_haplo","2":"160","3":"1094","4":"arg_t","5":"0.306891","6":"3.9484"}],"options":{"columns":{"min":{},"max":[10]},"rows":{"min":[10],"max":[10]},"pages":{}}}
  </script>

</div>

In [14]:
%%R
# write_csv(multiblup_thresholds, "reports/multiblup_thresholds.csv")

NULL


## Identify significant pathways

In [15]:
%%R
multiblup_result <- multiblup_summary %>%
  left_join(., gblup_llik) %>%
  left_join(., multiblup_llik) %>% 
  left_join(., gblup_summary) %>% 
  group_by(trait, pathway) %>%
  summarize_if(is.numeric, mean) %>% 
  ungroup() %>% 
  mutate(multiblup_lr = 2*(multiblup_llik - gblup_llik)) %>%
  left_join(., multiblup_thresholds, by = c("trait", "pathway")) %>%
  mutate(lr_pass = multiblup_lr > lr_95,
         h2_pass = multiblup_h2 > h2_95,
         pa_pass = (multiblup_pa - gblup_pa > 0.01),
         pathway = fct_recode(pathway,
                              aa_deg_old = "aa_degradation",
                              aa_syn_old = "aa_synthesis",
                              aa_degradation = "aa_deg_bcat_haplo",
                              aa_synthesis = "aa_syn_nobcat_haplo"
                              )) %>%
  filter(!pathway %in% exclude)

R[write to console]: Error in multiblup_summary %>% left_join(., gblup_llik) %>% left_join(.,  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in multiblup_summary %>% left_join(., gblup_llik) %>% left_join(.,  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [16]:
## Joining, by = "trait"

## Joining, by = c("trait", "pathway")

## Joining, by = c("trait", "cvnum", "fold", "gblup_h2")

In [17]:
%%R
pa_5 <- multiblup_result %>%
  mutate(pa_diff = multiblup_pa - gblup_pa) %>%
  filter(pa_diff >= 0.05,
         lr_pass == TRUE,
         h2_pass == TRUE,
         !pathway %in% exclude)

multiblup_pass <- multiblup_result %>%
  filter(lr_pass == TRUE,
         h2_pass == TRUE,
         pa_pass == TRUE) 

table3 <- multiblup_pass %>% 
  inner_join(aa_fam, ., by = c("trait")) %>%
  mutate(pa_diff = multiblup_pa - gblup_pa,
         reliability_diff = multiblup_reliability - gblup_reliability,
         int_diff = abs(multiblup_bias_int) - abs(gblup_bias_int),
         slope_diff = abs(multiblup_bias_slope - 1) - abs(gblup_bias_slope - 1),
         MSE_diff = multiblup_mse - gblup_mse) %>%
  mutate(
    aa_cat = fct_relevel(aa_cat, "absolute", "relative", "family"),
    category = ifelse(
      grepl("degradation_|protein_|protease", pathway),
      "protein",
      ifelse(
        grepl(
          "flavonoids|isoprenoids|N_containing|phenylpropanoids|S_containing",
          pathway
        ),
        "specialized",
        ifelse(
          grepl("aa_degradation|aa_synthesis|aa_transport", pathway),
          "amino acid",
          "primary"
        )
      )
    ) ,
    category = factor(
      category,
      levels = c("amino acid", "primary", "specialized", "protein")
    )
  ) %>%
  select(aa_cat, category, pathway, trait, h2_95, multiblup_h2, lr_95, multiblup_lr, 
         pa_diff, reliability_diff, int_diff, slope_diff, MSE_diff) %>%
  distinct() %>% 
  arrange(category, pathway, aa_cat) %>%
  write_csv("reports/table3.csv")

R[write to console]: Error in multiblup_result %>% mutate(pa_diff = multiblup_pa - gblup_pa) %>%  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in multiblup_result %>% mutate(pa_diff = multiblup_pa - gblup_pa) %>%  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [18]:
%%R
table3 %>% 
  kable(digits = 4) %>%
  kable_styling(bootstrap_options = "striped")

R[write to console]: Error in table3 %>% kable(digits = 4) %>% kable_styling(bootstrap_options = "striped") : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in table3 %>% kable(digits = 4) %>% kable_styling(bootstrap_options = "striped") : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


<table class="table table-striped" style="margin-left: auto; margin-right: auto;">

<thead>

<tr>

<th style="text-align:left;">

aa\_cat

</th>

<th style="text-align:left;">

category

</th>

<th style="text-align:left;">

pathway

</th>

<th style="text-align:left;">

trait

</th>

<th style="text-align:right;">

h2\_95

</th>

<th style="text-align:right;">

multiblup\_h2

</th>

<th style="text-align:right;">

lr\_95

</th>

<th style="text-align:right;">

multiblup\_lr

</th>

<th style="text-align:right;">

pa\_diff

</th>

<th style="text-align:right;">

reliability\_diff

</th>

<th style="text-align:right;">

int\_diff

</th>

<th style="text-align:right;">

slope\_diff

</th>

<th style="text-align:right;">

MSE\_diff

</th>

</tr>

</thead>

<tbody>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

his\_t

</td>

<td style="text-align:right;">

0.1894

</td>

<td style="text-align:right;">

0.2646

</td>

<td style="text-align:right;">

4.4288

</td>

<td style="text-align:right;">

5.6466

</td>

<td style="text-align:right;">

0.0267

</td>

<td style="text-align:right;">

0.0091

</td>

<td style="text-align:right;">

\-0.0003

</td>

<td style="text-align:right;">

\-0.0453

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

ile\_t

</td>

<td style="text-align:right;">

0.2835

</td>

<td style="text-align:right;">

0.4301

</td>

<td style="text-align:right;">

4.2966

</td>

<td style="text-align:right;">

9.1164

</td>

<td style="text-align:right;">

0.0666

</td>

<td style="text-align:right;">

0.0690

</td>

<td style="text-align:right;">

0.0002

</td>

<td style="text-align:right;">

\-0.0540

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

leu\_t

</td>

<td style="text-align:right;">

0.2196

</td>

<td style="text-align:right;">

0.2358

</td>

<td style="text-align:right;">

4.0528

</td>

<td style="text-align:right;">

5.1738

</td>

<td style="text-align:right;">

0.0315

</td>

<td style="text-align:right;">

0.0303

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0433

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

met\_t

</td>

<td style="text-align:right;">

0.1527

</td>

<td style="text-align:right;">

0.2787

</td>

<td style="text-align:right;">

3.3046

</td>

<td style="text-align:right;">

6.6332

</td>

<td style="text-align:right;">

0.0340

</td>

<td style="text-align:right;">

0.0246

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0090

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

his\_GluFam

</td>

<td style="text-align:right;">

0.2217

</td>

<td style="text-align:right;">

0.4197

</td>

<td style="text-align:right;">

4.6986

</td>

<td style="text-align:right;">

8.7320

</td>

<td style="text-align:right;">

0.0559

</td>

<td style="text-align:right;">

0.0301

</td>

<td style="text-align:right;">

0.0557

</td>

<td style="text-align:right;">

0.0834

</td>

<td style="text-align:right;">

0.8683

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

ile\_BCAA

</td>

<td style="text-align:right;">

0.1869

</td>

<td style="text-align:right;">

0.3551

</td>

<td style="text-align:right;">

4.4078

</td>

<td style="text-align:right;">

5.9804

</td>

<td style="text-align:right;">

0.0381

</td>

<td style="text-align:right;">

0.0337

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0227

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_degradation

</td>

<td style="text-align:left;">

val\_BCAA

</td>

<td style="text-align:right;">

0.1853

</td>

<td style="text-align:right;">

0.3432

</td>

<td style="text-align:right;">

3.5552

</td>

<td style="text-align:right;">

9.9922

</td>

<td style="text-align:right;">

0.0684

</td>

<td style="text-align:right;">

0.0452

</td>

<td style="text-align:right;">

0.0001

</td>

<td style="text-align:right;">

0.0418

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_synthesis

</td>

<td style="text-align:left;">

BCAA

</td>

<td style="text-align:right;">

0.1475

</td>

<td style="text-align:right;">

0.2901

</td>

<td style="text-align:right;">

4.1816

</td>

<td style="text-align:right;">

11.5750

</td>

<td style="text-align:right;">

0.0332

</td>

<td style="text-align:right;">

0.0315

</td>

<td style="text-align:right;">

0.0003

</td>

<td style="text-align:right;">

\-0.0256

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_synthesis

</td>

<td style="text-align:left;">

ile

</td>

<td style="text-align:right;">

0.2385

</td>

<td style="text-align:right;">

0.3907

</td>

<td style="text-align:right;">

3.5366

</td>

<td style="text-align:right;">

7.6928

</td>

<td style="text-align:right;">

0.0484

</td>

<td style="text-align:right;">

0.0443

</td>

<td style="text-align:right;">

0.0001

</td>

<td style="text-align:right;">

\-0.0428

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_synthesis

</td>

<td style="text-align:left;">

leu

</td>

<td style="text-align:right;">

0.1949

</td>

<td style="text-align:right;">

0.2564

</td>

<td style="text-align:right;">

3.2498

</td>

<td style="text-align:right;">

12.5570

</td>

<td style="text-align:right;">

0.0314

</td>

<td style="text-align:right;">

0.0325

</td>

<td style="text-align:right;">

0.0003

</td>

<td style="text-align:right;">

\-0.0212

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_synthesis

</td>

<td style="text-align:left;">

PyrFam

</td>

<td style="text-align:right;">

0.1649

</td>

<td style="text-align:right;">

0.3721

</td>

<td style="text-align:right;">

3.4310

</td>

<td style="text-align:right;">

5.1716

</td>

<td style="text-align:right;">

0.0315

</td>

<td style="text-align:right;">

0.0284

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0891

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

amino acid

</td>

<td style="text-align:left;">

aa\_transport

</td>

<td style="text-align:left;">

ShikFam

</td>

<td style="text-align:right;">

0.1011

</td>

<td style="text-align:right;">

0.2653

</td>

<td style="text-align:right;">

3.5514

</td>

<td style="text-align:right;">

6.4286

</td>

<td style="text-align:right;">

0.0286

</td>

<td style="text-align:right;">

0.0296

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0072

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

primary

</td>

<td style="text-align:left;">

e\_alt\_oxidase

</td>

<td style="text-align:left;">

arg\_t

</td>

<td style="text-align:right;">

0.0276

</td>

<td style="text-align:right;">

0.1604

</td>

<td style="text-align:right;">

3.9484

</td>

<td style="text-align:right;">

4.3464

</td>

<td style="text-align:right;">

0.0198

</td>

<td style="text-align:right;">

0.0041

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0149

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

primary

</td>

<td style="text-align:left;">

e\_alt\_oxidase

</td>

<td style="text-align:left;">

arg\_GluFam

</td>

<td style="text-align:right;">

0.1373

</td>

<td style="text-align:right;">

0.7726

</td>

<td style="text-align:right;">

3.3996

</td>

<td style="text-align:right;">

9.3904

</td>

<td style="text-align:right;">

0.0484

</td>

<td style="text-align:right;">

0.0453

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0669

</td>

<td style="text-align:right;">

0.0032

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

primary

</td>

<td style="text-align:left;">

glycolysis

</td>

<td style="text-align:left;">

ile\_t

</td>

<td style="text-align:right;">

0.2320

</td>

<td style="text-align:right;">

0.2858

</td>

<td style="text-align:right;">

3.7816

</td>

<td style="text-align:right;">

5.1886

</td>

<td style="text-align:right;">

0.0237

</td>

<td style="text-align:right;">

0.0207

</td>

<td style="text-align:right;">

\-0.0001

</td>

<td style="text-align:right;">

\-0.0712

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

flavonoids

</td>

<td style="text-align:left;">

pro

</td>

<td style="text-align:right;">

0.1026

</td>

<td style="text-align:right;">

0.1848

</td>

<td style="text-align:right;">

2.9836

</td>

<td style="text-align:right;">

3.4646

</td>

<td style="text-align:right;">

0.0165

</td>

<td style="text-align:right;">

0.0181

</td>

<td style="text-align:right;">

\-0.0003

</td>

<td style="text-align:right;">

\-0.0011

</td>

<td style="text-align:right;">

\-0.0001

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

flavonoids

</td>

<td style="text-align:left;">

tyr

</td>

<td style="text-align:right;">

0.1634

</td>

<td style="text-align:right;">

0.2501

</td>

<td style="text-align:right;">

3.7506

</td>

<td style="text-align:right;">

5.7490

</td>

<td style="text-align:right;">

0.0208

</td>

<td style="text-align:right;">

0.0100

</td>

<td style="text-align:right;">

\-0.0001

</td>

<td style="text-align:right;">

\-0.0085

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

isoprenoids

</td>

<td style="text-align:left;">

val\_BCAA

</td>

<td style="text-align:right;">

0.2309

</td>

<td style="text-align:right;">

0.3261

</td>

<td style="text-align:right;">

3.5552

</td>

<td style="text-align:right;">

6.2566

</td>

<td style="text-align:right;">

0.0865

</td>

<td style="text-align:right;">

0.0541

</td>

<td style="text-align:right;">

0.0001

</td>

<td style="text-align:right;">

\-0.1250

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

N\_containing

</td>

<td style="text-align:left;">

arg

</td>

<td style="text-align:right;">

0.1430

</td>

<td style="text-align:right;">

0.2025

</td>

<td style="text-align:right;">

3.0680

</td>

<td style="text-align:right;">

3.6448

</td>

<td style="text-align:right;">

0.0106

</td>

<td style="text-align:right;">

0.0060

</td>

<td style="text-align:right;">

\-0.0001

</td>

<td style="text-align:right;">

0.0463

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

phenylpropanoids

</td>

<td style="text-align:left;">

lys\_AspFam

</td>

<td style="text-align:right;">

0.2301

</td>

<td style="text-align:right;">

0.4522

</td>

<td style="text-align:right;">

3.6744

</td>

<td style="text-align:right;">

3.9488

</td>

<td style="text-align:right;">

0.0325

</td>

<td style="text-align:right;">

0.0255

</td>

<td style="text-align:right;">

\-0.0001

</td>

<td style="text-align:right;">

0.0486

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

phenylpropanoids

</td>

<td style="text-align:left;">

tyr\_ShikFam

</td>

<td style="text-align:right;">

0.1808

</td>

<td style="text-align:right;">

0.5620

</td>

<td style="text-align:right;">

4.6134

</td>

<td style="text-align:right;">

10.8572

</td>

<td style="text-align:right;">

0.1078

</td>

<td style="text-align:right;">

0.0741

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0056

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

met

</td>

<td style="text-align:right;">

0.1389

</td>

<td style="text-align:right;">

0.3043

</td>

<td style="text-align:right;">

3.5782

</td>

<td style="text-align:right;">

5.4620

</td>

<td style="text-align:right;">

0.0290

</td>

<td style="text-align:right;">

0.0240

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0364

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

phe

</td>

<td style="text-align:right;">

0.1448

</td>

<td style="text-align:right;">

0.2139

</td>

<td style="text-align:right;">

3.6820

</td>

<td style="text-align:right;">

4.3812

</td>

<td style="text-align:right;">

0.0282

</td>

<td style="text-align:right;">

0.0286

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0062

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

ShikFam

</td>

<td style="text-align:right;">

0.1011

</td>

<td style="text-align:right;">

0.2097

</td>

<td style="text-align:right;">

3.5514

</td>

<td style="text-align:right;">

8.1756

</td>

<td style="text-align:right;">

0.0210

</td>

<td style="text-align:right;">

0.0234

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0102

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

gln\_t

</td>

<td style="text-align:right;">

0.5850

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

4.5614

</td>

<td style="text-align:right;">

5.5922

</td>

<td style="text-align:right;">

0.0591

</td>

<td style="text-align:right;">

0.1266

</td>

<td style="text-align:right;">

0.0001

</td>

<td style="text-align:right;">

\-0.2597

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

phe\_t

</td>

<td style="text-align:right;">

0.1288

</td>

<td style="text-align:right;">

0.1909

</td>

<td style="text-align:right;">

3.7804

</td>

<td style="text-align:right;">

4.6016

</td>

<td style="text-align:right;">

0.0183

</td>

<td style="text-align:right;">

0.0170

</td>

<td style="text-align:right;">

0.0002

</td>

<td style="text-align:right;">

\-0.0192

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

specialized

</td>

<td style="text-align:left;">

S\_containing

</td>

<td style="text-align:left;">

gln\_GluFam

</td>

<td style="text-align:right;">

0.3260

</td>

<td style="text-align:right;">

0.8023

</td>

<td style="text-align:right;">

3.4424

</td>

<td style="text-align:right;">

6.4062

</td>

<td style="text-align:right;">

0.0637

</td>

<td style="text-align:right;">

0.1177

</td>

<td style="text-align:right;">

\-0.0023

</td>

<td style="text-align:right;">

\-0.0520

</td>

<td style="text-align:right;">

0.0015

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

degradation\_ubiquitin

</td>

<td style="text-align:left;">

ala\_t

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

4.1898

</td>

<td style="text-align:right;">

5.5670

</td>

<td style="text-align:right;">

0.0431

</td>

<td style="text-align:right;">

0.0461

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.2037

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

degradation\_ubiquitin

</td>

<td style="text-align:left;">

val\_PyrFam

</td>

<td style="text-align:right;">

0.4674

</td>

<td style="text-align:right;">

0.6257

</td>

<td style="text-align:right;">

8.7642

</td>

<td style="text-align:right;">

13.5286

</td>

<td style="text-align:right;">

0.0467

</td>

<td style="text-align:right;">

0.0186

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0507

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_aa\_activation

</td>

<td style="text-align:left;">

gly

</td>

<td style="text-align:right;">

0.1210

</td>

<td style="text-align:right;">

0.2518

</td>

<td style="text-align:right;">

3.8042

</td>

<td style="text-align:right;">

3.9984

</td>

<td style="text-align:right;">

0.0154

</td>

<td style="text-align:right;">

0.0240

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0197

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_aa\_activation

</td>

<td style="text-align:left;">

gly\_t

</td>

<td style="text-align:right;">

0.1660

</td>

<td style="text-align:right;">

0.3790

</td>

<td style="text-align:right;">

3.8150

</td>

<td style="text-align:right;">

5.3754

</td>

<td style="text-align:right;">

0.0336

</td>

<td style="text-align:right;">

0.0487

</td>

<td style="text-align:right;">

\-0.0001

</td>

<td style="text-align:right;">

\-0.0305

</td>

<td style="text-align:right;">

\-0.0001

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_aa\_activation

</td>

<td style="text-align:left;">

gly\_SerFam

</td>

<td style="text-align:right;">

0.2722

</td>

<td style="text-align:right;">

0.5070

</td>

<td style="text-align:right;">

3.1232

</td>

<td style="text-align:right;">

11.3982

</td>

<td style="text-align:right;">

0.0549

</td>

<td style="text-align:right;">

0.0840

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0903

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_aa\_activation

</td>

<td style="text-align:left;">

ser\_SerFam

</td>

<td style="text-align:right;">

0.2763

</td>

<td style="text-align:right;">

0.5053

</td>

<td style="text-align:right;">

3.1622

</td>

<td style="text-align:right;">

11.0810

</td>

<td style="text-align:right;">

0.0540

</td>

<td style="text-align:right;">

0.0842

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0930

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_assembly

</td>

<td style="text-align:left;">

met

</td>

<td style="text-align:right;">

0.0649

</td>

<td style="text-align:right;">

0.1252

</td>

<td style="text-align:right;">

3.5782

</td>

<td style="text-align:right;">

5.3084

</td>

<td style="text-align:right;">

0.0288

</td>

<td style="text-align:right;">

0.0208

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0066

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_assembly

</td>

<td style="text-align:left;">

met\_t

</td>

<td style="text-align:right;">

0.0352

</td>

<td style="text-align:right;">

0.1160

</td>

<td style="text-align:right;">

3.3046

</td>

<td style="text-align:right;">

4.0472

</td>

<td style="text-align:right;">

0.0147

</td>

<td style="text-align:right;">

0.0064

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0074

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_folding

</td>

<td style="text-align:left;">

his

</td>

<td style="text-align:right;">

0.0893

</td>

<td style="text-align:right;">

0.1386

</td>

<td style="text-align:right;">

8.8636

</td>

<td style="text-align:right;">

9.5644

</td>

<td style="text-align:right;">

0.0206

</td>

<td style="text-align:right;">

0.0132

</td>

<td style="text-align:right;">

\-0.0012

</td>

<td style="text-align:right;">

0.0322

</td>

<td style="text-align:right;">

0.0005

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_folding

</td>

<td style="text-align:left;">

met

</td>

<td style="text-align:right;">

0.1389

</td>

<td style="text-align:right;">

0.1927

</td>

<td style="text-align:right;">

3.5782

</td>

<td style="text-align:right;">

3.6692

</td>

<td style="text-align:right;">

0.0103

</td>

<td style="text-align:right;">

0.0058

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.0277

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_folding

</td>

<td style="text-align:left;">

ser

</td>

<td style="text-align:right;">

0.1725

</td>

<td style="text-align:right;">

0.3651

</td>

<td style="text-align:right;">

3.5862

</td>

<td style="text-align:right;">

6.6308

</td>

<td style="text-align:right;">

0.0464

</td>

<td style="text-align:right;">

0.0396

</td>

<td style="text-align:right;">

0.0010

</td>

<td style="text-align:right;">

\-0.0576

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_folding

</td>

<td style="text-align:left;">

SerFam

</td>

<td style="text-align:right;">

0.1315

</td>

<td style="text-align:right;">

0.2313

</td>

<td style="text-align:right;">

4.1560

</td>

<td style="text-align:right;">

6.7502

</td>

<td style="text-align:right;">

0.0381

</td>

<td style="text-align:right;">

0.0349

</td>

<td style="text-align:right;">

0.0009

</td>

<td style="text-align:right;">

0.0082

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

absolute

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_folding

</td>

<td style="text-align:left;">

Total

</td>

<td style="text-align:right;">

0.0862

</td>

<td style="text-align:right;">

0.1768

</td>

<td style="text-align:right;">

8.4766

</td>

<td style="text-align:right;">

9.0646

</td>

<td style="text-align:right;">

0.0224

</td>

<td style="text-align:right;">

0.0129

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0228

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_glyco

</td>

<td style="text-align:left;">

thr\_AspFam

</td>

<td style="text-align:right;">

0.1401

</td>

<td style="text-align:right;">

0.1520

</td>

<td style="text-align:right;">

3.1734

</td>

<td style="text-align:right;">

5.1948

</td>

<td style="text-align:right;">

0.0473

</td>

<td style="text-align:right;">

0.0431

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.0154

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_postrans

</td>

<td style="text-align:left;">

arg\_t

</td>

<td style="text-align:right;">

0.9264

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

3.9484

</td>

<td style="text-align:right;">

4.4518

</td>

<td style="text-align:right;">

0.0165

</td>

<td style="text-align:right;">

0.0232

</td>

<td style="text-align:right;">

0.0001

</td>

<td style="text-align:right;">

0.0410

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_postrans

</td>

<td style="text-align:left;">

arg\_GluFam

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

1.0000

</td>

<td style="text-align:right;">

3.3996

</td>

<td style="text-align:right;">

4.1710

</td>

<td style="text-align:right;">

0.0255

</td>

<td style="text-align:right;">

0.0216

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

\-0.1381

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

family

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_postrans

</td>

<td style="text-align:left;">

glu\_GluFam

</td>

<td style="text-align:right;">

0.6309

</td>

<td style="text-align:right;">

0.7087

</td>

<td style="text-align:right;">

3.1056

</td>

<td style="text-align:right;">

3.2760

</td>

<td style="text-align:right;">

0.0180

</td>

<td style="text-align:right;">

0.0014

</td>

<td style="text-align:right;">

0.0000

</td>

<td style="text-align:right;">

0.1107

</td>

<td style="text-align:right;">

0.0000

</td>

</tr>

<tr>

<td style="text-align:left;">

relative

</td>

<td style="text-align:left;">

protein

</td>

<td style="text-align:left;">

protein\_targeting

</td>

<td style="text-align:left;">

glu\_t

</td>

<td style="text-align:right;">

0.2994

</td>

<td style="text-align:right;">

0.4376

</td>

<td style="text-align:right;">

3.3590

</td>

<td style="text-align:right;">

3.9770

</td>

<td style="text-align:right;">

0.0190

</td>

<td style="text-align:right;">

0.0208

</td>

<td style="text-align:right;">

\-0.0155

</td>

<td style="text-align:right;">

0.0272

</td>

<td style="text-align:right;">

\-0.0034

</td>

</tr>

</tbody>

</table>